# 🤖 Simulación de Conversaciones - STI Chatbot

Este notebook simula 4 conversaciones completas con el chatbot STI para validar el flujo según el CSV definido.

## Escenarios de Prueba:

1. **Usuario Anónimo** - Problema: "mi compu no enciende"
2. **Roberto** - Instalación de app en Stick TV
3. **Heber** - Configuración WAN en MikroTik
4. **Valeria** - Notebook no enciende (debe generar ticket WhatsApp)

---

## 📦 Importar Librerías

In [ ]:
import requests
import json
import time
from datetime import datetime
import pandas as pd
from typing import Dict, List, Optional

# Configuración
BASE_URL = "http://localhost:3001"
HEADERS = {
    "Content-Type": "application/json",
    "Origin": "http://localhost:3001"
}

# Colores para output
class Colors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

print("✅ Librerías importadas correctamente")
print(f"🌐 Base URL: {BASE_URL}")

## 🛠️ Clase de Simulación de Conversaciones

In [ ]:
class ChatSimulator:
    def __init__(self, base_url: str, headers: dict):
        self.base_url = base_url
        self.headers = headers
        self.session_id = None
        self.conversation_log = []
        self.current_stage = None
        
    def start_conversation(self):
        """Inicia una nueva conversación obteniendo el saludo"""
        print(f"\n{Colors.HEADER}{'='*80}{Colors.ENDC}")
        print(f"{Colors.BOLD}🚀 Iniciando nueva conversación...{Colors.ENDC}")
        print(f"{Colors.HEADER}{'='*80}{Colors.ENDC}\n")
        
        response = requests.get(f"{self.base_url}/api/greeting", headers=self.headers)
        data = response.json()
        
        if data.get('ok'):
            self.session_id = data.get('sessionId')
            self.current_stage = data.get('stage')
            
            print(f"{Colors.OKBLUE}🆔 Session ID: {self.session_id[:16]}...{Colors.ENDC}")
            print(f"{Colors.OKCYAN}📍 Stage: {self.current_stage}{Colors.ENDC}\n")
            print(f"{Colors.OKGREEN}🤖 Bot:{Colors.ENDC} {data.get('reply')}\n")
            
            if data.get('options'):
                print(f"{Colors.WARNING}📋 Opciones:{Colors.ENDC}")
                for opt in data['options']:
                    print(f"   • {opt.get('label', opt)}")
            
            self.log_interaction("SYSTEM", "GREETING", data.get('reply'), self.current_stage)
            return data
        else:
            print(f"{Colors.FAIL}❌ Error iniciando conversación{Colors.ENDC}")
            return None
    
    def send_message(self, text: str = None, button_value: str = None, button_label: str = None):
        """Envía un mensaje o presiona un botón"""
        payload = {}
        
        if button_value:
            payload = {
                "action": "button",
                "value": button_value,
                "label": button_label or button_value,
                "text": ""
            }
            user_display = f"[BOTÓN] {button_label or button_value}"
        else:
            payload = {
                "text": text
            }
            user_display = text
        
        print(f"\n{Colors.OKCYAN}👤 Usuario:{Colors.ENDC} {user_display}")
        
        response = requests.post(
            f"{self.base_url}/api/chat",
            headers=self.headers,
            json=payload
        )
        
        data = response.json()
        
        if data.get('ok'):
            prev_stage = self.current_stage
            self.current_stage = data.get('stage')
            
            print(f"{Colors.OKGREEN}🤖 Bot:{Colors.ENDC} {data.get('reply')}\n")
            
            if prev_stage != self.current_stage:
                print(f"{Colors.WARNING}📍 Transición: {prev_stage} → {self.current_stage}{Colors.ENDC}\n")
            
            if data.get('options'):
                print(f"{Colors.WARNING}📋 Opciones:{Colors.ENDC}")
                for opt in data['options']:
                    label = opt.get('label') if isinstance(opt, dict) else opt
                    print(f"   • {label}")
                print()
            
            self.log_interaction(user_display, prev_stage, data.get('reply'), self.current_stage)
            
            time.sleep(0.5)  # Simular tiempo de pensamiento
            return data
        else:
            print(f"{Colors.FAIL}❌ Error: {data.get('error', 'Unknown')}{Colors.ENDC}")
            return None
    
    def log_interaction(self, user_input: str, from_stage: str, bot_reply: str, to_stage: str):
        """Registra una interacción"""
        self.conversation_log.append({
            'timestamp': datetime.now().isoformat(),
            'user_input': user_input,
            'from_stage': from_stage,
            'bot_reply': bot_reply[:100] + '...' if len(bot_reply) > 100 else bot_reply,
            'to_stage': to_stage
        })
    
    def get_log_dataframe(self):
        """Retorna el log como DataFrame"""
        return pd.DataFrame(self.conversation_log)
    
    def print_summary(self):
        """Imprime un resumen de la conversación"""
        print(f"\n{Colors.HEADER}{'='*80}{Colors.ENDC}")
        print(f"{Colors.BOLD}📊 RESUMEN DE LA CONVERSACIÓN{Colors.ENDC}")
        print(f"{Colors.HEADER}{'='*80}{Colors.ENDC}\n")
        print(f"Session ID: {self.session_id}")
        print(f"Total de interacciones: {len(self.conversation_log)}")
        print(f"Stage final: {self.current_stage}\n")
        
        # Mostrar flujo de stages
        stages = [log['from_stage'] for log in self.conversation_log]
        stages.append(self.current_stage)
        unique_stages = []
        for stage in stages:
            if stage and (not unique_stages or unique_stages[-1] != stage):
                unique_stages.append(stage)
        
        print(f"{Colors.OKCYAN}Flujo de etapas:{Colors.ENDC}")
        print(" → ".join(unique_stages))
        print()

print("✅ Clase ChatSimulator creada")

## 🎬 Simulación 1: Usuario Anónimo - "Mi compu no enciende"

**Perfil:**
- Nombre: Prefiero no decirlo
- Problema: Mi computadora no enciende
- Flujo esperado: ASK_LANGUAGE → ASK_NAME → ASK_NEED → ASK_PROBLEM → BASIC_TESTS

In [ ]:
print(f"{Colors.HEADER}\n{'='*80}")
print(f"🎬 SIMULACIÓN 1: Usuario Anónimo - 'Mi compu no enciende'")
print(f"{'='*80}{Colors.ENDC}\n")

sim1 = ChatSimulator()
sim1.start_conversation()
time.sleep(1)

# Usuario selecciona español de Argentina
sim1.send_message(button_id="BTN_LANG_ES_AR")
time.sleep(1)

# Usuario prefiere no decir nombre
sim1.send_message(button_id="BTN_NO_NAME")
time.sleep(1)

# Usuario necesita ayuda con problema
sim1.send_message(button_id="BTN_HELP")
time.sleep(1)

# Usuario describe el problema
sim1.send_message(text="mi compu no enciende")
time.sleep(1)

# Usuario proporciona marca/modelo
sim1.send_message(text="es una notebook HP Pavilion")
time.sleep(1)

# Usuario realiza pruebas básicas según bot indique
sim1.send_message(button_id="BTN_TESTS_DONE")
time.sleep(1)

print(f"\n{Colors.OKGREEN}✅ Simulación 1 completada{Colors.ENDC}")
sim1.print_summary()

# Guardar log
df1 = sim1.get_log_dataframe()
print(f"\n{Colors.OKCYAN}📊 Log de interacciones:{Colors.ENDC}")
print(df1.to_string(index=False))

## 🎬 Simulación 2: Roberto - "Instalar app en Stick TV"

**Perfil:**
- Nombre: Roberto
- Problema: Necesita ayuda para instalar una aplicación en su Stick TV
- Flujo esperado: ASK_LANGUAGE → ASK_NAME → ASK_NEED → ASK_HOWTO_DETAILS → PROVIDE_INSTRUCTIONS

In [ ]:
print(f"{Colors.HEADER}\n{'='*80}")
print(f"🎬 SIMULACIÓN 2: Roberto - 'Instalar app en Stick TV'")
print(f"{'='*80}{Colors.ENDC}\n")

sim2 = ChatSimulator()
sim2.start_conversation()
time.sleep(1)

# Usuario selecciona español de España
sim2.send_message(button_id="BTN_LANG_ES_ES")
time.sleep(1)

# Usuario proporciona nombre
sim2.send_message(text="Roberto")
time.sleep(1)

# Usuario necesita aprender algo
sim2.send_message(button_id="BTN_TASK")
time.sleep(1)

# Usuario describe qué necesita aprender
sim2.send_message(text="necesito ayuda para instalar una app en mi stick tv")
time.sleep(1)

# Usuario confirma que recibió las instrucciones
sim2.send_message(button_id="BTN_SOLVED")
time.sleep(1)

print(f"\n{Colors.OKGREEN}✅ Simulación 2 completada{Colors.ENDC}")
sim2.print_summary()

# Guardar log
df2 = sim2.get_log_dataframe()
print(f"\n{Colors.OKCYAN}📊 Log de interacciones:{Colors.ENDC}")
print(df2.to_string(index=False))

## 🎬 Simulación 3: Heber - "Configurar WAN en MikroTik"

**Perfil:**
- Nombre: Heber
- Problema: Configurar conexión WAN en un router MikroTik (problema técnico avanzado)
- Flujo esperado: ASK_LANGUAGE → ASK_NAME → ASK_NEED → ASK_PROBLEM → BASIC_TESTS → ESCALATE

In [ ]:
print(f"{Colors.HEADER}\n{'='*80}")
print(f"🎬 SIMULACIÓN 3: Heber - 'Configurar WAN en MikroTik'")
print(f"{'='*80}{Colors.ENDC}\n")

sim3 = ChatSimulator()
sim3.start_conversation()
time.sleep(1)

# Usuario selecciona inglés
sim3.send_message(button_id="BTN_LANG_EN")
time.sleep(1)

# Usuario proporciona nombre
sim3.send_message(text="Heber")
time.sleep(1)

# Usuario necesita ayuda con problema
sim3.send_message(button_id="BTN_HELP")
time.sleep(1)

# Usuario describe problema técnico avanzado
sim3.send_message(text="asistencia para configurar una conexión wan en un microtik")
time.sleep(1)

# Usuario proporciona detalles del dispositivo
sim3.send_message(text="MikroTik RB750Gr3")
time.sleep(1)

# Usuario indica que las pruebas básicas no funcionaron
sim3.send_message(button_id="BTN_TESTS_FAIL")
time.sleep(1)

# Usuario acepta escalamiento
sim3.send_message(button_id="BTN_YES")
time.sleep(1)

print(f"\n{Colors.OKGREEN}✅ Simulación 3 completada{Colors.ENDC}")
sim3.print_summary()

# Guardar log
df3 = sim3.get_log_dataframe()
print(f"\n{Colors.OKCYAN}📊 Log de interacciones:{Colors.ENDC}")
print(df3.to_string(index=False))

## 🎬 Simulación 4: Valeria - "Notebook no enciende" → Ticket WhatsApp

**Perfil:**
- Nombre: Valeria
- Problema: Su notebook no enciende (debe generar ticket para WhatsApp)
- Flujo esperado: ASK_LANGUAGE → ASK_NAME → ASK_NEED → ASK_PROBLEM → BASIC_TESTS → ESCALATE → CREATE_TICKET → ENDED
- **Caso crítico:** Validar generación de ticket completo

In [ ]:
print(f"{Colors.HEADER}\n{'='*80}")
print(f"🎬 SIMULACIÓN 4: Valeria - 'Notebook no enciende' → Ticket WhatsApp")
print(f"{'='*80}{Colors.ENDC}\n")

sim4 = ChatSimulator()
sim4.start_conversation()
time.sleep(1)

# Usuario selecciona español de Argentina
sim4.send_message(button_id="BTN_LANG_ES_AR")
time.sleep(1)

# Usuario proporciona nombre
sim4.send_message(text="Valeria")
time.sleep(1)

# Usuario necesita ayuda con problema
sim4.send_message(button_id="BTN_HELP")
time.sleep(1)

# Usuario describe problema (dice "tu notebook" en lugar de "mi notebook")
sim4.send_message(text="tu notebook no enciende")
time.sleep(1)

# Usuario proporciona marca/modelo
sim4.send_message(text="Dell Inspiron 15")
time.sleep(1)

# Usuario indica que las pruebas básicas no funcionaron
sim4.send_message(button_id="BTN_TESTS_FAIL")
time.sleep(1)

# Usuario acepta escalamiento y proporciona datos para ticket WhatsApp
sim4.send_message(button_id="BTN_YES")
time.sleep(1)

# Usuario proporciona email
sim4.send_message(text="valeria@email.com")
time.sleep(1)

# Usuario proporciona teléfono
sim4.send_message(text="+54 9 11 1234-5678")
time.sleep(1)

print(f"\n{Colors.OKGREEN}✅ Simulación 4 completada (TICKET GENERADO){Colors.ENDC}")
sim4.print_summary()

# Guardar log
df4 = sim4.get_log_dataframe()
print(f"\n{Colors.OKCYAN}📊 Log de interacciones:{Colors.ENDC}")
print(df4.to_string(index=False))

## 📊 Análisis de Flow Audit

Cargar los datos de auditoría generados por el sistema de logging para validar que no hubo loops ni problemas en el flujo.

In [ ]:
import os

# Leer archivo de auditoría generado por flowLogger
audit_file = os.path.join('data', 'logs', 'flow-audit.csv')

if os.path.exists(audit_file):
    df_audit = pd.read_csv(audit_file, encoding='utf-8-sig')
    
    print(f"{Colors.HEADER}📋 RESUMEN DE AUDITORÍA{Colors.ENDC}")
    print(f"{'='*80}\n")
    
    # Total de interacciones
    print(f"{Colors.OKBLUE}Total de interacciones:{Colors.ENDC} {len(df_audit)}")
    
    # Sesiones únicas
    sessions = df_audit['SessionId'].unique()
    print(f"{Colors.OKBLUE}Sesiones simuladas:{Colors.ENDC} {len(sessions)}")
    
    # Detectar loops (misma etapa repetida 3+ veces consecutivas)
    loops_detected = []
    for session in sessions:
        session_data = df_audit[df_audit['SessionId'] == session]
        stages = session_data['Etapa Actual'].tolist()
        
        for i in range(len(stages) - 2):
            if stages[i] == stages[i+1] == stages[i+2]:
                loops_detected.append({
                    'session': session[:8],
                    'stage': stages[i],
                    'count': stages.count(stages[i])
                })
                break
    
    if loops_detected:
        print(f"\n{Colors.FAIL}⚠️  LOOPS DETECTADOS: {len(loops_detected)}{Colors.ENDC}")
        for loop in loops_detected:
            print(f"  - Session {loop['session']}: '{loop['stage']}' repetido {loop['count']} veces")
    else:
        print(f"\n{Colors.OKGREEN}✅ No se detectaron loops en el flujo{Colors.ENDC}")
    
    # Estadísticas por etapa
    print(f"\n{Colors.OKCYAN}📊 Distribución de etapas:{Colors.ENDC}")
    stage_counts = df_audit['Etapa Actual'].value_counts()
    for stage, count in stage_counts.items():
        print(f"  {stage}: {count}")
    
    # Duración promedio
    avg_duration = df_audit['Duración (ms)'].mean()
    print(f"\n{Colors.OKBLUE}⏱️  Duración promedio por interacción:{Colors.ENDC} {avg_duration:.2f} ms")
    
    # Mostrar últimas 10 interacciones
    print(f"\n{Colors.HEADER}📋 ÚLTIMAS 10 INTERACCIONES{Colors.ENDC}")
    print(df_audit[['SessionId', 'Etapa Actual', 'Input Usuario', 'Trigger Detectado', 'Siguiente Etapa']].tail(10).to_string(index=False))
    
else:
    print(f"{Colors.FAIL}❌ Archivo de auditoría no encontrado: {audit_file}{Colors.ENDC}")
    print(f"{Colors.WARNING}Asegúrate de que el servidor esté ejecutándose y que flowLogger.js esté activo.{Colors.ENDC}")